In [2]:

import torch
import torch.nn as nn
import torch.optim as optim

# Define a custom neural network class
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        
        # Define the network layers
        self.linear1 = nn.Linear(input_size, hidden_size) # First linear layer
        self.activation1 = nn.ReLU() # Activation function after the first layer
        self.linear2 = nn.Linear(hidden_size, output_size) # Second linear layer
        self.softmax = nn.Softmax(dim=1) # Softmax layer for the output
        self.logsoftmax = nn.LogSoftmax(dim=1) # LogSoftmax layer for the output
    
    def forward(self, x):
        # Define the forward pass
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.softmax(x)
        x = self.logsoftmax(x)
        return x

# Create an instance of the network
input_size = 10
hidden_size = 5
output_size = 3
model = SimpleNN(input_size, hidden_size, output_size)

# Create a loss function
criterion = nn.NLLLoss()

# Create an optimizer
# optimizer = optim.SGD(model.parameters(), lr=0.01)

# Define a batch of inputs and targets
batch_size = 4
input_data = torch.randn(batch_size, input_size) # Random input tensor with batch size
targets = torch.randint(0, output_size, (batch_size,)) # Random target tensor for each input in the batch

# Forward pass
outputs = model(input_data)

# Compute loss
loss = criterion(outputs, targets)

# Backward pass
# loss.backward()

# Update weights
# optimizer.step()

# Optionally, print output and loss
print(f"Outputs:\n{outputs}")
print(f"Loss: {loss.item()}")


Outputs:
tensor([[0.3025, 0.3291, 0.3683],
        [0.2412, 0.2542, 0.5046],
        [0.3385, 0.3532, 0.3083],
        [0.3237, 0.3334, 0.3428]], grad_fn=<SoftmaxBackward0>)
Loss: 1.0599757432937622


In [3]:
# random array and save it to a file .npy
# import numpy as np

# B = 1000
# N = 100
# M = 30
# X = np.random.rand(B, N).astype(np.float32)
# W = np.random.rand(M,N).astype(np.float32)
# bias = np.random.rand(M).astype(np.float32)
# np.save('../with-torch-tests/linear-layer/X.npy', X)
# np.save('../with-torch-tests/linear-layer/W.npy', W)
# np.save('../with-torch-tests/linear-layer/bias.npy', bias)

# l = nn.Linear(N,M)
# l.weight.data = torch.from_numpy(W)
# l.bias.data = torch.from_numpy(bias)
# X_torch = torch.from_numpy(X)
# Y = l(X_torch)
# np.save('../with-torch-tests/linear-layer/Y.npy', Y.detach().numpy())
# print(Y[0,0:5])


In [32]:
import numpy as np
import torch
import torch.nn as nn

# Load the input data
ln1b = np.load('../with-torch-tests/all-model/ln1b.npy')
ln1w = np.load('../with-torch-tests/all-model/ln1w.npy')
ln2b = np.load('../with-torch-tests/all-model/ln2b.npy')
ln2w = np.load('../with-torch-tests/all-model/ln2w.npy')
target = np.load('../with-torch-tests/all-model/target.npy')
X_c = np.load('../with-torch-tests/all-model/X_c.npy')

# get sizes
B,N = X_c.shape
H1,N = ln1w.shape
l1 = nn.Linear(N,H1)
l1.weight.data = torch.from_numpy(ln1w).to(torch.float32)
l1.bias.data = torch.from_numpy(ln1b).to(torch.float32)

H2,_ = ln2w.shape
l2 = nn.Linear(H1,H2)
l2.weight.data = torch.from_numpy(ln2w).to(torch.float32)
l2.bias.data = torch.from_numpy(ln2b).to(torch.float32)

X = torch.from_numpy(X_c).to(torch.float32)
Y1 = l1(X)
Y2 = l2(Y1)
print(Y2.shape)

criterion = nn.CrossEntropyLoss(reduction='sum')
loss = criterion(Y2, torch.from_numpy(target).long())
loss

torch.Size([2, 5])


tensor(4.7015, grad_fn=<NllLossBackward0>)

In [4]:
# read arrays from .npy files and func Linear to compare
import numpy as np
import torch
import torch.nn as nn

X = np.load('../with-torch-tests/linear-layer/X_C.npy')
W = np.load('../with-torch-tests/linear-layer/W_C.npy')
bias = np.load('../with-torch-tests/linear-layer/bias_C.npy')
Y = np.load('../with-torch-tests/linear-layer/out_C.npy')
B,N = X.shape
_,M = Y.shape
l = nn.Linear(M,N)
l.weight.data = torch.from_numpy(W).to(torch.float32)
l.bias.data = torch.from_numpy(bias).to(torch.float32)
X_torch = torch.from_numpy(X).to(torch.float32)
Y_torch = l(X_torch)

print(Y[0,0:5])
print(Y_torch[0,0:5])
print( np.allclose(Y, Y_torch.detach().numpy(), atol=1e-4, rtol=1e-4))



[-2.4500256  -4.164225    2.0214434   0.48262623  0.05314786]
tensor([-2.4500, -4.1642,  2.0214,  0.4826,  0.0531], grad_fn=<SliceBackward0>)
True


In [5]:
# Relu forward and backward tests
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Load data
X = np.load(r'../with-torch-tests/relu-layer/X_relu.npy')
Y = np.load(r'../with-torch-tests/relu-layer/out_relu.npy')
dY = np.load(r'../with-torch-tests/relu-layer/up_grad_relu.npy')
dX = np.load(r'../with-torch-tests/relu-layer/down_grad_relu.npy')
# Convert to PyTorch tensors
# make sure to set requires_grad=True for the input tensor so that the Autograd engine can compute the gradients
X_torch = torch.from_numpy(X).to(torch.float32).requires_grad_(True)
dY_torch = torch.from_numpy(dY).to(torch.float32)

# Forward pass with ReLU
relu = nn.ReLU()
Y_torch = relu(X_torch)

# Compare the forward pass results
print("Forward pass comparison:")
print("Y (numpy):", Y[0, 0:5])
print("Y_torch:", Y_torch.detach().numpy()[0, 0:5])
print("Match:", np.allclose(Y, Y_torch.detach().numpy(), atol=1e-4, rtol=1e-4))


# Validate the backward pass
Y_torch.backward(dY_torch)

# Get the gradients from X_torch
dX_torch = X_torch.grad

# Compare the backward pass results
print("\nBackward pass comparison:")
print("dX (numpy):", dX[0, 0:5])
print("dX_torch:", dX_torch.numpy()[0, 0:5])
print("Match:", np.allclose(dX, dX_torch.numpy(), atol=1e-4, rtol=1e-4))


Forward pass comparison:
Y (numpy): [0.09152496 0.         0.3774835  0.         0.59312725]
Y_torch: [0.09152496 0.         0.3774835  0.         0.59312725]
Match: True

Backward pass comparison:
dX (numpy): [-0.9976806   0.          0.29630423  0.         -0.45951718]
dX_torch: [-0.9976806   0.          0.29630423  0.         -0.45951718]
Match: True


In [7]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
print(target.dtype)
output = loss(m(input), target)
output.backward()
# 2D loss example (used, for example, with image inputs)
N, C = 5, 4
loss = nn.NLLLoss()
# input is of size N x C x height x width
data = torch.randn(N, 16, 10, 10)
conv = nn.Conv2d(16, C, (3, 3))
m = nn.LogSoftmax(dim=1)
# each element in target has to have 0 <= value < C
target = torch.empty(N, 8, 8, dtype=torch.long).random_(0, C)
output = loss(m(conv(data)), target)
output.backward()

torch.int64
